In [1]:
from decouple import config
import os
import uuid

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma

from langchain_ollama import ChatOllama
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from PDFChatBot import PDFChatBot

In [2]:
session_id = str(uuid.uuid4()).replace('-', '_')

In [3]:
if 'OLLAMA_API_BASE_URL' not in os.environ:
    os.environ["OPENAI_API_KEY"] = config('OPENAI_API_KEY')
OLLAMA_API_BASE_URL = os.environ['OLLAMA_API_BASE_URL'] if 'OLLAMA_API_BASE_URL' in os.environ else config('OLLAMA_API_BASE_URL')   
LLM = os.environ['LLM'] if 'LLM' in os.environ else config('LLM')   
EMBEDDING_MODEL = os.environ['EMBEDDING_MODEL'] if 'EMBEDDING_MODEL' in os.environ else config('EMBEDDING_MODEL')  

In [4]:
print(f'Using embedding model: {EMBEDDING_MODEL}')
embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

Using embedding model: sentence-transformers/all-MiniLM-L6-v2


/Users/stolli/miniforge3/envs/pdf-chat-bot/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/stolli/miniforge3/envs/pdf-chat-bot/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
print(f'Using LLM: {LLM}')
llm = ChatOllama(
    base_url=OLLAMA_API_BASE_URL, 
    model=LLM
)

Using LLM: llama3.1:8b


In [6]:
chat_bot = PDFChatBot('/Users/stolli/IT/Designing Data-Intensive Applications.pdf', embedding_model, llm)

incorrect startxref pointer(1)


Initializing PDF Chatbot ...
--- Loading and vectorizing PDF file ---


parsing for Object Streams


--- Initializing history aware retriever ---
--- Initializing Q & A chain ---
--- Initializing RAG chain ---


In [7]:
stream_response = []
for chunk in chat_bot.stream_response('What is partitioning?', session_id):
    stream_response.append(chunk)
    print(chunk, end="\n", flush=True)

--- Streaming response ---
{'input': 'What is partitioning?', 'chat_history': []}
{'context': [Document(metadata={'page': 233, 'source': '/Users/stolli/IT/Designing Data-Intensive Applications.pdf'}, page_content='both key-range and hash partitioning, and it splits partitions dynamically in either\ncase.\nPartitioning proportionally to nodes\nWith dynamic partitioning, the number of partitions is proportional to the size of the\ndataset, since the splitting and merging processes keep the size of each partition\nbetween some fixed minimum and maximum. On the other hand, with a fixed num‐\n212 | Chapter 6: Partitioning'), Document(metadata={'page': 233, 'source': '/Users/stolli/IT/Designing Data-Intensive Applications.pdf'}, page_content='both key-range and hash partitioning, and it splits partitions dynamically in either\ncase.\nPartitioning proportionally to nodes\nWith dynamic partitioning, the number of partitions is proportional to the size of the\ndataset, since the splitting and m

In [8]:
''.join([chunk['answer'] for chunk in stream_response if 'answer' in chunk.keys()])

'Partitioning refers to the process of dividing a dataset or data storage into smaller, more manageable pieces (partitions) based on specific criteria such as range, hash, or other methods. This can be done for various purposes like improving performance, reducing memory usage, or enhancing scalability in databases, distributed systems, and data processing applications.'

In [9]:
response = chat_bot.get_response('What is the book about? Please summarize it in around 20 sentences. Include a list of the most important topics', session_id=session_id)

--- Generating response ---


In [10]:
print(response['answer'])

The book "Designing Data-Intensive Applications" by Martin Kleppmann is about designing and building scalable and reliable large-scale data systems.

Here's a summary:

The book starts with an overview of how data has become increasingly important in modern computing, and how traditional systems are no longer sufficient to handle the scale and complexity of today's data-driven applications. The author then dives into the details of designing and implementing data-intensive systems, covering topics such as data modeling, storage, and processing.

The book emphasizes the importance of thinking about data as a first-class citizen in system design, rather than an afterthought. It also highlights the need for a deep understanding of the underlying technology and the trade-offs involved in making design choices.

The author discusses various approaches to storing and retrieving data, including relational databases, NoSQL databases, and distributed storage systems like Apache Cassandra and Am

In [11]:
response['context']

[Document(metadata={'page': 608, 'source': '/Users/stolli/IT/Designing Data-Intensive Applications.pdf'}, page_content='Lamport, 345\nlogical, 494\nordering events, 291, 345\nTitan (database), 50\ntombstones, 74, 191, 456\ntopics (messaging), 137, 440\ntotal order, 341, 557\nlimits of, 493\nsequence numbers or timestamps, 344\ntotal order broadcast, 348-352, 493, 522\nconsensus algorithms and, 366-368\nIndex | 587'),
 Document(metadata={'page': 612, 'source': '/Users/stolli/IT/Designing Data-Intensive Applications.pdf'}, page_content='About the Author\nMartin Kleppmann  is a researcher in distributed systems at the University of Cam‐\nbridge, UK. Previously he was a software engineer and entrepreneur at internet com‐\npanies including LinkedIn and Rapportive, where he worked on large-scale data\ninfrastructure. In the process he learned a few things the hard way, and he hopes this\nbook will save you from repeating the same mistakes.\nMartin is a regular conference speaker, blogger, an

In [17]:
res = []
[res.append(x) for x in response['context'] if x not in res]
res

[Document(metadata={'page': 608, 'source': '/Users/stolli/IT/Designing Data-Intensive Applications.pdf'}, page_content='Lamport, 345\nlogical, 494\nordering events, 291, 345\nTitan (database), 50\ntombstones, 74, 191, 456\ntopics (messaging), 137, 440\ntotal order, 341, 557\nlimits of, 493\nsequence numbers or timestamps, 344\ntotal order broadcast, 348-352, 493, 522\nconsensus algorithms and, 366-368\nIndex | 587'),
 Document(metadata={'page': 612, 'source': '/Users/stolli/IT/Designing Data-Intensive Applications.pdf'}, page_content='About the Author\nMartin Kleppmann  is a researcher in distributed systems at the University of Cam‐\nbridge, UK. Previously he was a software engineer and entrepreneur at internet com‐\npanies including LinkedIn and Rapportive, where he worked on large-scale data\ninfrastructure. In the process he learned a few things the hard way, and he hopes this\nbook will save you from repeating the same mistakes.\nMartin is a regular conference speaker, blogger, an

In [12]:
response['context'][0].page_content

'Lamport, 345\nlogical, 494\nordering events, 291, 345\nTitan (database), 50\ntombstones, 74, 191, 456\ntopics (messaging), 137, 440\ntotal order, 341, 557\nlimits of, 493\nsequence numbers or timestamps, 344\ntotal order broadcast, 348-352, 493, 522\nconsensus algorithms and, 366-368\nIndex | 587'

In [13]:
for document in response['context']:
    print(f'Source: {document.metadata["source"]}')
    print(f'Page: {document.metadata["page"]}')
    print(f'Content: {document.page_content}\n')


Source: /Users/stolli/IT/Designing Data-Intensive Applications.pdf
Page: 608
Content: Lamport, 345
logical, 494
ordering events, 291, 345
Titan (database), 50
tombstones, 74, 191, 456
topics (messaging), 137, 440
total order, 341, 557
limits of, 493
sequence numbers or timestamps, 344
total order broadcast, 348-352, 493, 522
consensus algorithms and, 366-368
Index | 587

Source: /Users/stolli/IT/Designing Data-Intensive Applications.pdf
Page: 612
Content: About the Author
Martin Kleppmann  is a researcher in distributed systems at the University of Cam‐
bridge, UK. Previously he was a software engineer and entrepreneur at internet com‐
panies including LinkedIn and Rapportive, where he worked on large-scale data
infrastructure. In the process he learned a few things the hard way, and he hopes this
book will save you from repeating the same mistakes.
Martin is a regular conference speaker, blogger, and open source contributor. He
believes that profound technical ideas should be accessib

In [14]:
# chat_bot.get_response('What is partitioning?', session_id=session_id)

In [15]:
# chat_bot.get_response('Can you repeat the answer as structured list?', session_id=session_id)